In [1]:
# Import data

In [1]:
import torch
from torch.utils.data import Dataset
from torchvision import transforms
from hevisum_dataset import importDataset
import numpy as np


/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/torchvision/io/image.py:14: UserWarning: Failed to load image Python extension: 'dlopen(/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/torchvision/image.so, 0x0006): Library not loaded: @rpath/libjpeg.9.dylib
  Referenced from: <FB2FD416-6C4D-3621-B677-61F07C02A3C5> /opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/torchvision/image.so
  Reason: tried: '/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/opt/anaconda3/envs/spatialhackathon/lib/python3.9/lib-dynload/../../libjpeg.9.dylib' (no such file), '/opt/anaconda3/envs/spatialhackathon/bin/../lib/libjpeg.9.dylib' (no such file)'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might b

In [2]:
from sklearn.preprocessing import StandardScaler
import numpy as np
import torch
from hevisum_dataset import importDataset

# 載入資料
img_data = torch.load("../train_dataset.pt")
graph_data = torch.load("../train_graph_dataset.pt")

/var/folders/7g/j5s0yzcj34l3v043s7znkplc0000gn/T/ipykernel_2755/235426519.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  img_data = torch.load("../train_dataset.pt")
/v

In [ ]:
# 正確方式

raw_coords = []
for _meta in img_data['meta_info']:
    if _meta is not None:
        _, x, y = _meta
    else:
        x, y = 0, 0
    raw_coords.append([x, y])

raw_coords = np.array(raw_coords)


coord_scaler = StandardScaler()
normalized_coords = coord_scaler.fit_transform(raw_coords)

# ====== Step 2: 建立 Dataset ======
train_dataset = importDataset(
    center_tile=img_data['tiles'],
    subtiles=img_data['subtiles'],
    neighbor_tiles=img_data['neighbor_tiles'],
    label=img_data['labels'],
    meta=normalized_coords,
    # 👇 加上 graph 資料
    node_feat=graph_data['node_feats'],
    adj_lists=graph_data['adj_lists'],
    edge_feats=graph_data['edge_feats']
)

In [39]:
graph_data['adj_lists'].shape

AttributeError: 'list' object has no attribute 'shape'

In [28]:
def check_dataset_item(dataset, idx=0):
    item = dataset[idx]

    print("🔍 Checking dataset sample:", idx)

    # --- Center Tile ---
    tile = item['center_tile']
    print(f"📏 Center tile shape: {tile.shape} | dtype: {tile.dtype} | min: {tile.min():.3f}, max: {tile.max():.3f}")
    assert tile.ndim == 3 and tile.shape[0] == 3, "❌ Center tile shape 不正確，應為 (3, H, W)"

    # --- Subtiles ---
    subtiles = item['subtiles']
    print(f"📏 Subtiles shape: {subtiles.shape} | dtype: {subtiles.dtype}")
    assert subtiles.shape[0] == 9 and subtiles.shape[1] == 3, "❌ Subtile shape 不正確，應為 (9, 3, h, w)"

    # --- Neighbors ---
    neighbors = item['neighbor_tiles']
    print(f"📏 Neighbor tiles shape: {neighbors.shape} | dtype: {neighbors.dtype}")
    assert neighbors.shape[0] == 8 and neighbors.shape[1] == 3, "❌ Neighbor tile shape 不正確，應為 (8, 3, H, W)"

    # --- Label ---
    label = item['label']
    print(f"🧬 Label shape: {label.shape} | dtype: {label.dtype}")
    assert label.shape[0] == 35 and label.dtype == torch.float32, "❌ Label 應為 float32 且長度為 35"

    # --- Coordinates (meta) ---
    coordinates = item['meta']
    print(f"📌 Coordinates (meta): x = {coordinates[0]}, y = {coordinates[1]}")
    assert isinstance(coordinates, torch.Tensor) and coordinates.shape == (2,), "❌ meta 應為 shape (2,) 的 tensor"

    # --- Node Features ---
    if 'node_feat' in item and item['node_feat'] is not None:
        node_feat = item['node_feat']
        print(f"📊 Node feature shape: {node_feat.shape} | dtype: {node_feat.dtype}")
        assert node_feat.ndim == 1, "❌ Node feature 應為 1D tensor"
    else:
        print("⚠️ Node features 未提供")

    # --- Edge Features ---
    if 'edge_feat' in item and item['edge_feat'] is not None:
        edge_feat = item['edge_feat']
        print(f"🔗 Edge feature shape: {edge_feat.shape} | dtype: {edge_feat.dtype}")
        print(f"🔗 Sample edge_feat[0]: {edge_feat[0]}")
        assert edge_feat.ndim == 2 and edge_feat.shape[1] == 5, "❌ Edge feature 應為 (k, 5)"
    else:
        print("⚠️ Edge features 未提供")

    # --- Adjacency List ---
    if 'adj_list' in item and item['adj_list'] is not None:
        adj_list = item['adj_list']
        print(f"📎 Adjacency list shape: {adj_list.shape} | dtype: {adj_list.dtype}")
        print(f"📎 Sample: {adj_list[:3]}")
        assert adj_list.ndim == 2 and adj_list.shape[1] == 2, "❌ Adjacency list 應為 (k, 2)"
    else:
        print("⚠️ Adjacency list 未提供")

    print("✅ All checks passed!")




    
check_dataset_item(train_dataset)


🔍 Checking dataset sample: 0
📏 Center tile shape: torch.Size([3, 78, 78]) | dtype: torch.float32 | min: -0.192, max: 0.953
📏 Subtiles shape: torch.Size([9, 3, 26, 26]) | dtype: torch.float32
📏 Neighbor tiles shape: torch.Size([8, 3, 78, 78]) | dtype: torch.float32
🧬 Label shape: torch.Size([35]) | dtype: torch.float32
📌 Coordinates (meta): x = 1.3731348514556885, y = 0.7239122986793518
📊 Node feature shape: torch.Size([14]) | dtype: torch.float32
🔗 Edge feature shape: torch.Size([7, 5]) | dtype: torch.float32
🔗 Sample edge_feat[0]: tensor([26.0000, 26.0000,  0.0000,  0.0000,  0.0385])
📎 Adjacency list shape: torch.Size([7, 2]) | dtype: torch.float32
📎 Sample: tensor([[6.2800e+02, 3.8462e-02],
        [2.3700e+02, 3.8462e-02],
        [1.3880e+03, 3.7851e-02]])
✅ All checks passed!


/var/folders/7g/j5s0yzcj34l3v043s7znkplc0000gn/T/ipykernel_2755/3104603402.py:74: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'node_feat': torch.tensor(self.node_feats[idx], dtype=torch.float32) if self.node_feats is not None else None,


In [29]:
from torch.utils.data import DataLoader, random_split

# 設定比例
train_ratio = 0.8
val_ratio = 1 - train_ratio
total_len = len(train_dataset)
train_len = int(train_ratio * total_len)
val_len = total_len - train_len

# 拆分 Dataset
generator = torch.Generator().manual_seed(42)
train_set, val_set = random_split(train_dataset, [train_len, val_len], generator=generator)

print(f"✅ Train: {len(train_set)} samples")
print(f"✅ Val: {len(val_set)} samples")

# 🔹 將其包成 DataLoader
BATCH_SIZE = 32

train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=False)


✅ Train: 6679 samples
✅ Val: 1670 samples


Poteintial issues:
# 1. my val_set tiles image may be included in the sub_tiles of train_set

Note: Since neighbor tiles are reused across samples, some mild information overlap may exist between train and val sets. However, final test set is completely held out.

# Neural Network

In [62]:
import torch
import torch.nn as nn
from torch_geometric.nn import GATConv  # 確保你有導入這個模組

# GAT Encoder：將 node_feats, adj_lists, edge_feats → 局部 GAT 表徵
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch
import torch.nn as nn
import torch.nn.functional as F

import torch
import torch.nn as nn
import torch.nn.functional as F

import torch
import torch.nn as nn
import torch.nn.functional as F

class GATModel(nn.Module):
    def __init__(self, in_dim=14, out_dim=128, heads=4, dropout_rate=0.2):
        super(GATModel, self).__init__()

        self.in_dim = in_dim
        self.out_dim = out_dim
        self.heads = heads
        self.dropout_rate = dropout_rate
        
        # MLP 層：將節點特徵進行初步變換
        self.mlp = nn.Sequential(
            nn.Linear(in_dim, 64),
            nn.ReLU(),
            nn.Linear(64, out_dim)  # 輸入 GAT 層所需的維度
        )
        
        # 初始化多個注意力頭的權重
        self.W = nn.Parameter(torch.zeros(heads, out_dim, out_dim))  # (heads, out_dim, out_dim)
        self.a1 = nn.Parameter(torch.zeros(heads, out_dim, 1))  # Attention weight 1
        self.a2 = nn.Parameter(torch.zeros(heads, out_dim, 1))  # Attention weight 2

    def forward(self, x, adj_lists, edge_feats):
        """
        x: node_feats, (N, in_dim) → 節點特徵
        adj_lists: (B, E, 2) → 每個圖的鄰接列表 [B, E, 2]
        edge_feats: (B, E, edge_feat_dim) → 邊特徵
        """
        # Step 1: 使用 MLP 進行節點特徵處理
        x = self.mlp(x)  # 經過 MLP 之後 (N, out_dim)

        # Step 2: 計算圖注意力，這裡我們使用 `gat_attention_layer` 來處理每個圖
        multi_head_output = self.gat_attention_layer(x, adj_lists, edge_feats)
        return multi_head_output  # (N, out_dim)

    def gat_attention_layer(self, node_features, adj_lists, edge_feats):
        """
        自定義圖注意力機制，根據邊特徵調整注意力分數
        """
        num_nodes = node_features.size(0)
        
        # Step 3: 使用多頭注意力進行計算
        multi_head_output = torch.zeros((num_nodes, self.heads * self.out_dim)).to(node_features.device)
        
        for head in range(self.heads):
            attention_scores_head = torch.zeros((num_nodes, num_nodes)).to(node_features.device)

            # Step 4: 計算每個節點對鄰居的注意力，使用自定義計算的權重
            for i in range(num_nodes):
                for j in adj_lists[i].long():  # .long() 確保 j 是 int64 類型
                    attention_scores_head[i, j] = self.compute_attention_score(node_features[i], node_features[j], edge_feats[i], head)

            # 使用 softmax 計算注意力權重
            attention_scores_head = F.softmax(attention_scores_head, dim=1)

            # 根據注意力權重更新節點特徵
            weighted_sum_head = torch.matmul(attention_scores_head, node_features)
            multi_head_output[:, head*self.out_dim:(head+1)*self.out_dim] = weighted_sum_head

        return multi_head_output

    def compute_attention_score(self, node_i_features, node_j_features, edge_feats, head):
        """
        計算節點間的注意力分數
        """
        # 計算節點特徵間的相似度，這裡使用 torch.matmul 來處理 2D 張量
        distance_score = torch.matmul(node_i_features, node_j_features.T)  # 使用 .T 轉置，使之成為點積

        # 取得邊的特徵（邊的距離或權重）
        edge_weight = edge_feats[0][0]  # 假設邊權重來自邊特徵
        
        # 計算注意力分數，這裡將邊特徵影響納入考慮
        attention_score = distance_score * edge_weight
        
        # 使用 LeakyReLU 激活函數來處理非線性部分（這部分在原模型中有）
        attention_score = F.leaky_relu(attention_score, negative_slope=0.01)
        
        return attention_score







class CNNEncoder(nn.Module):
    def __init__(self, out_dim):
        super().__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d((1, 1))
        )
        self.flatten = nn.Flatten()
        self.linear = nn.Linear(64, out_dim)

    def forward(self, x):  # x: (B, 3, H, W)
        x = self.cnn(x)     # → (B, 64, 1, 1)
        x = self.flatten(x) # → (B, 64)
        x = self.linear(x)  # → (B, out_dim)
        return x

class MLPDecoder(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.mlp = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, output_dim)  # 👉 Linear activation
        )

    def forward(self, x):
        return self.mlp(x)
    
class VisionGraphFusionModel(nn.Module):
    def __init__(self, cnn_out_dim=64, gat_out_dim=128, output_dim=35):
        super().__init__()
        self.encoder_spot = CNNEncoder(cnn_out_dim)
        self.encoder_subtiles = CNNEncoder(cnn_out_dim)
        self.encoder_neighbors = CNNEncoder(cnn_out_dim)

        # 局部 GAT Encoder
        self.gat_encoder = GATEncoder(in_dim=14, out_dim=gat_out_dim)

        # concat: 3×cnn + gat
        self.decoder = MLPDecoder(input_dim=cnn_out_dim * 3 + gat_out_dim, output_dim=output_dim)

    def forward(self, center_tile, subtiles, neighbor_tiles, node_feats, adj_lists, edge_feats):
        """
        node_feats:     (N, 14)         → 所有 node 的 features
        adj_lists:      (E, 2)          → 局部鄰接列表，形狀 [邊數, 2]
        edge_feats:     (E, edge_feat_dim) → 邊的特徵
        """
        B = center_tile.size(0)

        # Image encoders
        f_center = self.encoder_spot(center_tile)  # (B, D)

        B, N, C, h, w = subtiles.shape
        subtiles = subtiles.view(B * N, C, h, w)
        f_sub = self.encoder_subtiles(subtiles).view(B, N, -1).mean(dim=1)  # (B, D)

        B, N, C, H, W = neighbor_tiles.shape
        neighbor_tiles = neighbor_tiles.view(B * N, C, H, W)
        f_neigh = self.encoder_neighbors(neighbor_tiles).view(B, N, -1).mean(dim=1)  # (B, D)

        # Graph encoder (局部圖處理)
        f_graph = self.gat_encoder(node_feats, adj_lists, edge_feats)  # (N, 128)

        # Fuse everything
        x = torch.cat([f_center, f_sub, f_neigh, f_graph], dim=1)  # (B, 3D + GAT)
        return self.decoder(x)  # (B, 35)
    

In [63]:
import torch
import numpy as np
from scipy.stats import spearmanr
from tqdm import tqdm

# 🧠 訓練一個 epoch
def train_one_epoch(model, dataloader, optimizer, loss_fn, device):
    model.train()
    total_loss = 0
    pbar = tqdm(dataloader, desc="Training", leave=False)

    for batch in pbar:
        center = batch['center_tile'].to(device)
        subtiles = batch['subtiles'].to(device)
        neighbors = batch['neighbor_tiles'].to(device)
        label = batch['label'].to(device)

        # Fetch graph data
        node_feats = batch['node_feat'].to(device)  # (N, 14)
        adj_lists = batch['adj_list'].to(device)  # (E, 2), 局部鄰接列表
        edge_feats = batch['edge_feat'].to(device)  # (E, edge_feat_dim), 邊特徵

        optimizer.zero_grad()
        out = model(center, subtiles, neighbors, node_feats, adj_lists, edge_feats)
        loss = loss_fn(out, label)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * center.size(0)
        avg_loss = total_loss / ((pbar.n + 1) * dataloader.batch_size)
        pbar.set_postfix(loss=loss.item(), avg=avg_loss)

    return total_loss / len(dataloader.dataset)

# 📏 驗證模型
def evaluate(model, dataloader, loss_fn, device):
    model.eval()
    total_loss = 0
    preds, targets = [], []

    pbar = tqdm(dataloader, desc="Evaluating", leave=False)

    with torch.no_grad():
        for batch in pbar:
            center = batch['center_tile'].to(device)
            subtiles = batch['subtiles'].to(device)
            neighbors = batch['neighbor_tiles'].to(device)
            label = batch['label'].to(device)

            # Fetch graph data
            node_feats = batch['node_feat'].to(device)  # (N, 14)
            adj_lists = batch['adj_list'].to(device)  # (E, 2), 局部鄰接列表
            edge_feats = batch['edge_feat'].to(device)  # (E, edge_feat_dim), 邊特徵

            out = model(center, subtiles, neighbors, node_feats, adj_lists, edge_feats)
            loss = loss_fn(out, label)

            total_loss += loss.item() * center.size(0)
            preds.append(out.cpu())
            targets.append(label.cpu())

            pbar.set_postfix(loss=loss.item())

    preds = torch.cat(preds).numpy()
    targets = torch.cat(targets).numpy()

    # ✅ Spearman correlation for each gene
    scores = [spearmanr(preds[:, i], targets[:, i])[0] for i in range(preds.shape[1])]
    spearman_avg = np.nanmean(scores)

    return total_loss / len(dataloader.dataset), spearman_avg

# 🔮 預測
def predict(model, dataloader, device):
    model.eval()
    all_preds = []
    all_meta = []

    with torch.no_grad():
        for batch in dataloader:
            center = batch['center_tile'].to(device)
            subtiles = batch['subtiles'].to(device)
            neighbors = batch['neighbor_tiles'].to(device)
            # Fetch graph data
            node_feats = batch['node_feat'].to(device)  # (N, 14)
            adj_lists = batch['adj_list'].to(device)  # (E, 2), 局部鄰接列表
            edge_feats = batch['edge_feat'].to(device)  # (E, edge_feat_dim), 邊特徵

            out = model(center, subtiles, neighbors, node_feats, adj_lists, edge_feats)
            all_preds.append(out.cpu())
            all_meta.extend(batch['meta'])

    return torch.cat(all_preds).numpy(), all_meta


# callback

In [64]:
class EarlyStopping:
    def __init__(self, patience=10, verbose=True):
        self.patience = patience
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.verbose = verbose

    def __call__(self, val_loss):
        if self.best_score is None or val_loss < self.best_score:
            self.best_score = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.verbose:
                print(f"EarlyStopping counter: {self.counter}/{self.patience}")
            if self.counter >= self.patience:
                self.early_stop = True


import matplotlib.pyplot as plt
from IPython.display import clear_output


def plot_losses(train_losses, val_losses):
    clear_output(wait=True)
    plt.figure(figsize=(8, 4))
    plt.plot(train_losses, label="Train Loss", marker='o')
    plt.plot(val_losses, label="Val Loss", marker='o')
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()
    plt.grid(True)
    plt.title("Training vs Validation Loss")
    plt.show()
# 收集資料

In [ ]:
import torch
import csv
import matplotlib.pyplot as plt
from IPython.display import clear_output
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm import tqdm

# 🔧 設定裝置
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"✅ Using device: {device}")

# 🔧 初始化模型 & 優化器
model = VisionGraphFusionModel().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)
loss_fn = nn.MSELoss()
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)
early_stopper = EarlyStopping(patience=10)

# 🔧 儲存 log 的設定
log_file = open("training_log.csv", mode="w", newline="")
csv_writer = csv.writer(log_file)
csv_writer.writerow(["Epoch", "Train Loss", "Val Loss", "Val Spearman", "Learning Rate"])

# 🔧 用來畫圖
train_losses = []
val_losses = []


# 🔁 開始訓練
num_epochs = 150
best_val_loss = float('inf')

for epoch in range(num_epochs):
    train_loss = train_one_epoch(model, train_loader, optimizer, loss_fn, device)
    val_loss, val_spearman = evaluate(model, val_loader, loss_fn, device)

    # ✅ 儲存最好的模型
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), "best_model.pt")
        print("✅ Saved best model!")

    # ✅ 調整學習率
    scheduler.step(val_loss)

    # ✅ 寫入 CSV log
    lr = optimizer.param_groups[0]['lr']
    csv_writer.writerow([epoch+1, train_loss, val_loss, val_spearman, lr])

    # ✅ 印 epoch 結果
    print(f"Epoch {epoch+1}/{num_epochs} | loss: {train_loss:.4f} | val_loss: {val_loss:.4f} | ρ: {val_spearman:.4f} | lr: {lr:.2e}")

    # ✅ 更新 loss list 並畫圖
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    plot_losses(train_losses, val_losses)

    # ✅ Early stopping
    early_stopper(val_loss)
    if early_stopper.early_stop:
        print("⛔ Early stopping triggered.")
        break

# ✅ 關閉 log 檔案
log_file.close()


✅ Using device: mps


Training:   0%|          | 0/209 [00:00<?, ?it/s]/var/folders/7g/j5s0yzcj34l3v043s7znkplc0000gn/T/ipykernel_2755/3104603402.py:74: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'node_feat': torch.tensor(self.node_feats[idx], dtype=torch.float32) if self.node_feats is not None else None,
Training:   6%|▌         | 12/209 [00:04<00:50,  3.90it/s, avg=nan, loss=nan]       

＃# Prediction

In [59]:
# ===== 需要的 Libraries =====
import torch
import pandas as pd
import numpy as np
from tqdm import tqdm
from torch.utils.data import DataLoader
import csv

# ===== 載入訓練好的模型權重 =====
from hevisum_model import HEVisumModel
from hevisum_dataset import importDataset

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"✅ Using device: {device}")

model = VisionMLPModelWithCoord().to(device)
model.load_state_dict(torch.load("best_model.pt"))
model.eval()


✅ Using device: mps


/var/folders/7g/j5s0yzcj34l3v043s7znkplc0000gn/T/ipykernel_54542/3135847424.py:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_model

VisionMLPModelWithCoord(
  (encoder_spot): CNNEncoder(
    (cnn): Sequential(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (4): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (5): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (6): ReLU()
      (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (8): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (9): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (10): ReLU()
      (11): AdaptiveAvgPool2d(output_size=(1, 1))
    )
    (flatten): Flatten(start_dim=1, end_dim=-1)
    (linear): Linear(in_features=64, out_features=64, bias=True)
  )
  (encoder_subtiles

In [61]:
# 正確方式
test_data = torch.load("test_dataset.pt")

raw_coords = []
for _meta in data['meta_info']:
    if _meta is not None:
        _, x, y = _meta
    else:
        x, y = 0, 0
    raw_coords.append([x, y])

raw_coords = np.array(raw_coords)


coord_scaler = StandardScaler()
normalized_coords = coord_scaler.fit_transform(raw_coords)

# ====== Step 2: 建立 Dataset ======
test_dataset = importDataset(
    center_tile=test_data['tiles'],
    subtiles=test_data['subtiles'],
    neighbor_tiles=test_data['neighbor_tiles'],
    label=np.zeros((len(test_data['tiles']), 35)),  # dummy label
    meta=normalized_coords
)


test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


/var/folders/7g/j5s0yzcj34l3v043s7znkplc0000gn/T/ipykernel_54542/1343659088.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  test_data = torch.load("test_dataset.pt")


In [62]:
test_preds, test_meta = predict(model, test_loader, device)


coords.shape = torch.Size([32, 2])
coords.shape = torch.Size([32, 2])
coords.shape = torch.Size([32, 2])
coords.shape = torch.Size([32, 2])
coords.shape = torch.Size([32, 2])
coords.shape = torch.Size([32, 2])
coords.shape = torch.Size([32, 2])
coords.shape = torch.Size([32, 2])
coords.shape = torch.Size([32, 2])
coords.shape = torch.Size([32, 2])
coords.shape = torch.Size([32, 2])
coords.shape = torch.Size([32, 2])
coords.shape = torch.Size([32, 2])
coords.shape = torch.Size([32, 2])
coords.shape = torch.Size([32, 2])
coords.shape = torch.Size([32, 2])
coords.shape = torch.Size([32, 2])
coords.shape = torch.Size([32, 2])
coords.shape = torch.Size([32, 2])
coords.shape = torch.Size([32, 2])
coords.shape = torch.Size([32, 2])
coords.shape = torch.Size([32, 2])
coords.shape = torch.Size([32, 2])
coords.shape = torch.Size([32, 2])
coords.shape = torch.Size([32, 2])
coords.shape = torch.Size([32, 2])
coords.shape = torch.Size([32, 2])
coords.shape = torch.Size([32, 2])
coords.shape = torch

In [ ]:
test_preds

array([[0.29158217, 0.12338457, 0.17179191, ..., 0.03849616, 0.01655579,
        0.02754541],
       [0.29158217, 0.12338457, 0.17179191, ..., 0.03849616, 0.01655579,
        0.02754541],
       [0.25503415, 0.13337179, 0.1524582 , ..., 0.04139816, 0.01808124,
        0.03039141],
       ...,
       [0.29158217, 0.12338457, 0.17179191, ..., 0.03849616, 0.01655579,
        0.02754541],
       [0.29158217, 0.12338457, 0.17179191, ..., 0.03849616, 0.01655579,
        0.02754541],
       [0.23385419, 0.13244098, 0.13941698, ..., 0.04153137, 0.01910294,
        0.03005139]], dtype=float32)

In [66]:
import h5py

# ==== 讀取 test spot index 用於對應 ID ====
with h5py.File("./elucidata_ai_challenge_data.h5", "r") as f:
    test_spots = f["spots/Test"]
    test_spot_table = pd.DataFrame(np.array(test_spots['S_7']))  # Example: S_7


ensemble_df = pd.DataFrame(test_preds, columns=[f"C{i+1}" for i in range(test_preds.shape[1])])
ensemble_df.insert(0, 'ID', test_spot_table.index)
ensemble_df.to_csv("submission.csv", index=False)
print("✅ Saved submission.csv")


✅ Saved submission.csv
